
# KOR-END 번역 모델 만들기

이번 특강에서는 한국어 문장을 영어로 번역하는 시퀀스-투-시퀀스(sequence-to-sequence, seq2seq) 모델을 학습하는 방법을 알아보겠습니다.

필요 라이브러리: ``torchtext``, ``spacy`` 를 사용하여 데이터셋을 전처리(preprocess)합니다.

## Import

#### Download Requirements

In [ ]:
!pip install --upgrade git+https://github.com/dAiv-CNU/torchdaiv.git

  Cloning https://github.com/dAiv-CNU/torchdaiv.git to /tmp/pip-req-build-9ihb754w
  Running command git clone --filter=blob:none --quiet https://github.com/dAiv-CNU/torchdaiv.git /tmp/pip-req-build-9ihb754w
  Resolved https://github.com/dAiv-CNU/torchdaiv.git to commit adb140010d1ebafdf87c8dfa0805f2f27f2fff2e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download ko_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 17.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ko_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


#### Library Imports

In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import spacy
from torchdaiv import datasets
from torchdaiv.lectures.kor_eng_translator import nn
from torchdaiv.lectures.kor_eng_translator.util import vocabulary, transforms

from rich.traceback import install
#install(show_locals=True)  # 오류 났을 경우 로컬 변수 보고 싶으면 활성화

%matplotlib inline

---
## Text Preprocess (with Spacy)

``torchtext`` 에는 언어 변환 모델을 만들 때 쉽게 사용할 수 있는 데이터셋을 만들기 적합한 다양한 도구가 있습니다.
이 예제에서는 가공되지 않은 텍스트 문장(raw text sentence)을 토큰화(tokenize)하고, 어휘집(vocabulary)을 만들고,
토큰을 텐서로 숫자화(numericalize)하는 방법을 알아보겠습니다.

| (다만, torchtext는 2024년 4월 이후 더 이상 업데이트가 진행되지 않는다는 점에 유의해야 합니다.)

아래를 실행하여 Spacy 토크나이저가 쓸 한국어와 영어에 대한 데이터를 다운로드 받습니다.

In [2]:
# spacy tokenizer 적용
ko_tokenizer = vocabulary.load_tokenizer(spacy, "ko_core_news_sm")
en_tokenizer = vocabulary.load_tokenizer(spacy, "en_core_web_sm")

In [3]:
from spacy.lang.ko.examples import sentences

# 작동 확인
doc = spacy.load("ko_core_news_sm")(sentences[0])
print("Original:", doc.text)
print("Tokenized:", ko_tokenizer(sentences[0]), end="\n\n")

for token in doc:
    print(">", token.text, f"({token.lemma_}) |", token.pos_, token.dep_)

Original: 애플이 영국의 스타트업을 10억 달러에 인수하는 것을 알아보고 있다.
Tokenized: ['애플', '##이', '영국', '##의', '스타트업', '##을', '10', '##억', '달러', '##에', '인수', '##하', '##는', '것', '##을', '알아보', '##고', '있', '##다', '.']

> 애플이 (애플+이) | NOUN dislocated
> 영국의 (영국+의) | PROPN nmod
> 스타트업을 (스타트업+을) | NOUN nsubj
> 10억 (10+억) | NUM compound
> 달러에 (달러+에) | ADV obl
> 인수하는 (인수+하+는) | VERB acl
> 것을 (것+을) | NOUN obj
> 알아보고 (알아보+고) | AUX ROOT
> 있다 (있+다) | AUX aux
> . (.) | PUNCT punct


---
## Load Dataset
using spacy

In [4]:
# 데이터셋 로드 - 아무 처리도 하지 않았을 때
# 이번 수업에서는 트레인 데이터셋으로만 사용

train_dataset = datasets.AnkiKorEngDataset("./data", split_rate=(1.0, 0.0, 0.0))
# valid_dataset = datasets.AnkiKorEngDataset("./data", valid=True, split_rate=(0.5, 0.3, 0.2))
# test_dataset = datasets.AnkiKorEngDataset("./data", test=True, split_rate=(0.5, 0.3, 0.2))

Extraction completed.
Dataset loaded. 5822 samples loaded.


In [5]:
# 데이터셋 형태 확인
sample = list(zip(*train_dataset[0:5]))+list(zip(*train_dataset[500:505]))
for i, (kor, eng) in enumerate(sample):
    print(i, kor, eng)

0 가. Go.
1 안녕. Hi.
2 뛰어! Run!
3 뛰어. Run.
4 누구? Who?
5 저건 뭐야? What's that?
6 누가 그를 그렸습니까? Who drew it?
7 쉬엄쉬엄 일해. Work slowly.
8 너 늦었어. You're late.
9 넌 내 거야. You're mine.


#### Vocabulary 생성

In [6]:
ko_vocab = vocabulary.build_vocab(raw_dataset=train_dataset.raw_kor, tokenizer=ko_tokenizer)
en_vocab = vocabulary.build_vocab(raw_dataset=train_dataset.raw_eng, tokenizer=en_tokenizer)

#### Convert To Tensor

In [7]:
# 사전 데이터를 기반으로 데이터셋을 텐서로 변환
to_tensor = (
    transforms.to_tensor(ko_vocab, tokenizer=ko_tokenizer),
    transforms.to_tensor(en_vocab, tokenizer=en_tokenizer)
)

train_dataset.transform(transform=to_tensor)
# valid_dataset.transform(transform=to_tensor)
# test_dataset.transform(transform=to_tensor)

Using Special Tokens - PAD_IDX: 0, UNK_IDX: 1
Using Special Tokens - PAD_IDX: 0, UNK_IDX: 1


In [8]:
# 데이터셋 형태 확인
sample = list(zip(*train_dataset[0:5]))+list(zip(*train_dataset[500:505]))
for i, (kor, eng) in enumerate(sample):
    print(i, kor, eng)

0 tensor([52,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0]) tensor([25,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  

#### Data Loader

In [9]:
# 배치 크기 결정 후 데이터 로더 생성
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# valid_dataload = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset)//20)

## Model Definition
> RNN/GRU 레이어를 하나만 사용하였던 지난 주차와는 달리, 레이어를 여러 층으로 쌓는 방식의 Encoder와 Decoder 모델을 사용

> 인코더는 한국어를 해석하고, 디코더는 영어를 생성하는 방식으로 역할을 나눠서 번역을 수행


> 참고사항:
>> 아래 예시 모델은 공부하기 쉬운 단순한 모델로 번역에 있어 매우 뛰어난 성능을 보이는 모델은 아닙니다.
>> 최신 기술 트렌드는 Transformers를 사용하는 것입니다.
>> 혹시 관심이 있다면 [Transformer 레이어](https://pytorch.org/docs/stable/nn.html#transformer-layers)를 사용하는 코드로 변경해서 진행해보기 바랍니다.

In [10]:
# 사전 등록
nn.set_vocabulary(ko_vocab, en_vocab)

5497 2535 2535


In [11]:
# 모델 정의
class Seq2Seq(nn.Module):
    def __init__(self, height=2, hidden=128):
        super(Seq2Seq, self).__init__()
        self.encoder = nn.Encoder(nn.GRU, height=height, hidden=hidden, dropout=0.3)
        self.decoder = nn.Decoder(nn.GRU, height=height, hidden=hidden, dropout=0.3)

    def forward(self, korean, english):
        context_vector = self.encoder(korean)
        output = self.decoder(english, context_vector)
        return output

In [12]:
# 하이퍼 파라미터 설정
epoch = 20
lr = 1e-4  # learning rate
height = 2
hidden = 128

In [13]:
# 모델 생성
model = Seq2Seq(height=height, hidden=hidden)
model.init_optimizer(lr=lr)
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5497, 256)
    (model): GRU(256, 128, num_layers=2, batch_first=True, dropout=0.3)
  )
  (decoder): Decoder(
    (embedding): Embedding(2535, 256)
    (model): GRU(256, 128, num_layers=2, batch_first=True, dropout=0.3)
    (fc1): Linear(in_features=128, out_features=1267, bias=True)
    (fc2): Linear(in_features=1267, out_features=2535, bias=True)
  )
  (criterion): CrossEntropyLoss()
)

In [28]:
# 가능한 경우 쿠다 사용
if torch.cuda.is_available:
    model.cuda()
print("Use Device:", model.device)

Use Device: cuda


In [15]:
model.fit(train_dataloader, epoch)

epoch:1/20 loss: 0.2035551503464416
epoch:2/20 loss: 0.19860664255671448
epoch:3/20 loss: 0.19753132203778068
epoch:4/20 loss: 0.19665333399405846
epoch:5/20 loss: 0.19568360326709328
epoch:6/20 loss: 0.1950620723949684
epoch:7/20 loss: 0.1947883264376567
epoch:8/20 loss: 0.19457984085266405
epoch:9/20 loss: 0.19440441865187424
epoch:10/20 loss: 0.1942486998799083
epoch:11/20 loss: 0.1940966906783345
epoch:12/20 loss: 0.19396408475362337
epoch:13/20 loss: 0.19386575657587785
epoch:14/20 loss: 0.19370876694773578
epoch:15/20 loss: 0.19361033757309337
epoch:16/20 loss: 0.1935026493373808
epoch:17/20 loss: 0.19341186867965446


KeyboardInterrupt: 

## Translation Test

In [29]:
model.translate('좋은 아침!', transform=transforms.to_tensor(ko_vocab, tokenizer=ko_tokenizer))

Using Special Tokens - PAD_IDX: 0, UNK_IDX: 1
understand the understand . 